In [786]:
import os;
import json;
import builtins
import pandas as pd;

dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge'
testcase = 'join_fb_1'
test_dir = os.path.join(dir, testcase)
results_dir = os.path.join(test_dir, 'outputs', 'results')
csv_dir = os.path.join(test_dir, 'csv')
os.makedirs(csv_dir, exist_ok=True)

runs = [os.path.join(results_dir, run) for run in os.listdir(results_dir)]
print(runs)

['/home/chesetti/Repos/learned_merge_cleanup/sponge/join_fb_1/outputs/results/run']


In [787]:
# Walk all the json files and put them in a dataframe

test_results = []
for run in runs:
    for test_result_file in os.listdir(run):
        json_file = builtins.open(os.path.join(run, test_result_file))
        test_result = json.load(json_file)
        test_result['run'] = run
        test_results.append(test_result)
        json_file.close()
test_dataframe = pd.json_normalize(test_results)
print(test_dataframe.columns)

        

Index(['command', 'run', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_index_build_duration_ns',
       'result.inner_index_size', 'result.inner_total_bytes_fetched',
       'result.outer_disk_fetch', 'result.outer_disk_fetch_size',
       'result.outer_total_bytes_fetched', 'spec.algo', 'spec.algo_name',
       'spec.common_key', 'spec.index.search', 'spec.index.type',
       'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk', 'spec.index.leaf_size_in_pages'],
      dtype='object')


# Reports

### Overall Duration

In [796]:
overall_duration = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.duration_ns', aggfunc='median')
overall_duration.to_csv(os.path.join(csv_dir, 'duration_sec.csv'))
print(overall_duration.columns)
print(overall_duration.index)
print(overall_duration)
throughput = overall_duration

for column in throughput.columns:
    throughput[column] = (200_000_000/throughput.index) * (1000_000_000.0 / throughput[column])
print(throughput)
overall_duration[["btree2048", "btree1024", "btree256"]].plot() 
overall_duration[["pgm128", "pgm512", "pgm1024"]].plot() 

Index(['btree1024', 'btree2048', 'btree256', 'hashJoin', 'pgm1024', 'pgm128',
       'pgm512', 'sj'],
      dtype='object', name='spec.algo_name')
Index([10, 20, 30, 40, 50, 60, 70, 80, 90, 100], dtype='int64', name='spec.common_key')
spec.algo_name      btree1024     btree2048      btree256      hashJoin  \
spec.common_key                                                           
10               4.708207e+09  4.486402e+09  5.109228e+09  4.769367e+10   
20               2.462185e+09  2.636967e+09  3.192786e+09  4.566060e+10   
30               1.790857e+09  1.717178e+09  2.219935e+09  3.786643e+10   
40               1.714835e+09  1.384545e+09  1.589914e+09  4.252691e+10   
50               1.845927e+09  1.487593e+09  1.386194e+09  3.781501e+10   
60               1.098073e+09  1.062159e+09  1.232999e+09  3.442171e+10   
70               9.999145e+08  1.245923e+09  1.714164e+09  4.073743e+10   
80               1.192378e+09  8.921607e+08  1.671640e+09  3.743386e+10   
90             

NameError: name 'overa' is not defined

In [789]:
def parseIndexName(name):
    index = name
    epsilon = 0
    if name=="btree1024":
        index="BTree"
        epsilon=1024
    if name=="btree2048":
        index="BTree"
        epsilon=2048
    if name=="btree256":
        index="BTree"
        epsilon=256
    if name=="pgm1024":
        index="PGM"
        epsilon=2049
    if name=="pgm512":
        index="PGM"
        epsilon=1025
    if name=="pgm128":
        index="PGM"
        epsilon=257
    return {"name": index, "epsilon": epsilon}


inner_index_size = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_index_size', aggfunc='median')
data_by_epsilon = []
data = {}
data['indexes'] = []
data['memory_bytes'] = []
for index in inner_index_size:
    data['indexes'].append(index)
    data['memory_bytes'].append(inner_index_size[index].mean())
    index_dict = parseIndexName(index)
    index_dict['memory'] = inner_index_size[index].mean()
    data_by_epsilon.append(index_dict)
inner_index_size = pd.DataFrame(data=data)
inner_index_size.to_csv(os.path.join(csv_dir, 'inner_index_size.csv'))
print(inner_index_size)
inner_index_size_by_eps = pd.DataFrame(data_by_epsilon).pivot(index='epsilon', columns='name',values='memory')
inner_index_size_by_eps.to_csv(os.path.join(csv_dir, 'inner_index_size_by_eps.csv'))

     indexes  memory_bytes
0  btree1024     3160008.0
1  btree2048     1586176.0
2   btree256    12627696.0
3   hashJoin    35019496.0
4    pgm1024      296992.0
5     pgm128     4143880.0
6     pgm512      810544.0
7         sj    35019496.0


In [790]:
inner_index_build_duration = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_index_build_duration_ns', aggfunc='median')
data_by_epsilon = []
data = {}
data['indexes'] = []
data['build_duration'] = []
for index in inner_index_build_duration:
    data['indexes'].append(index)
    data['build_duration'].append(inner_index_build_duration[index].mean())
    index_dict = parseIndexName(index)
    index_dict['build_duration'] = inner_index_build_duration[index].mean()
    data_by_epsilon.append(index_dict)
inner_index_build_duration = pd.DataFrame(data=data)
inner_index_build_duration.to_csv(os.path.join(csv_dir, 'inner_index_build_duration_ns.csv'))
print(inner_index_build_duration)
inner_index_bd_by_eps = pd.DataFrame(data_by_epsilon).pivot(index='epsilon', columns='name',values='build_duration')
inner_index_bd_by_eps.to_csv(os.path.join(csv_dir, 'inner_index_size_by_eps.csv'))
print(inner_index_bd_by_eps)

     indexes  build_duration
0  btree1024    1.977777e+09
1  btree2048    1.954453e+09
2   btree256    1.976906e+09
3   hashJoin    2.303432e+10
4    pgm1024    1.830953e+10
5     pgm128    2.115134e+10
6     pgm512    1.955904e+10
7         sj    2.304843e+10
name            BTree           PGM      hashJoin            sj
epsilon                                                        
0                 NaN           NaN  2.303432e+10  2.304843e+10
256      1.976906e+09           NaN           NaN           NaN
257               NaN  2.115134e+10           NaN           NaN
1024     1.977777e+09           NaN           NaN           NaN
1025              NaN  1.955904e+10           NaN           NaN
2048     1.954453e+09           NaN           NaN           NaN
2049              NaN  1.830953e+10           NaN           NaN


In [791]:
inner_index_fetch = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_disk_fetch', aggfunc='median')
inner_index_total_fetch = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_total_bytes_fetched', aggfunc='median')
inner_index_fetch_size = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_disk_fetch_size', aggfunc='median')
print(inner_index_fetch)
print(inner_index_total_fetch)
print(inner_index_fetch_size)

spec.algo_name   btree1024  btree2048  btree256  hashJoin  pgm1024  pgm128  \
spec.common_key                                                              
10                  195313      97657    781250    781250    86805  390455   
20                  195313      97657    781250    781250    86803  389806   
30                  195313      97657    781099    781250    86800  388612   
40                  195313      97657    779956    781250    86793  386887   
50                  195313      97657    776542    781250    86786  384625   
60                  195313      97657    770178    781250    86775  381852   
70                  195313      97657    761014    781250    86763  378580   
80                  195313      97657    748922    781250    86747  374994   
90                  195312      97657    735895    781250    86729  371108   
100                 195308      97657    720931    781250    86708  367094   

spec.algo_name   pgm512      sj  
spec.common_key              

In [792]:
result_checksum = (test_dataframe[['spec.common_key', 'spec.algo_name', 'result.checksum']].sort_values(by=['spec.common_key', 'spec.algo_name'])) #.loc[test_dataframe['spec.common_key']=='10'])
for common_key in sorted(test_dataframe['spec.common_key'].unique()):
    checksums = result_checksum.loc[result_checksum['spec.common_key'] == common_key]
    unique_checksums = checksums['result.checksum'].unique()
    if (len(unique_checksums) == 1):
        print(f"common_key: {common_key} checksum: {unique_checksums} OK")
    else:
        print(f"common_key: {common_key} checksums don't match")
print(result_checksum.loc[result_checksum['spec.common_key'] == 50])
print(result_checksum.loc[result_checksum['spec.common_key'] == 50]['result.checksum'].unique())

common_key: 10 checksum: ['193D20B4D79BEBE03A8D1175D7D436A7'] OK
common_key: 20 checksum: ['00488473353CFD3EA9C88895C7676366'] OK
common_key: 30 checksum: ['6A442C89930EFC8ADDED12F53F502BEE'] OK
common_key: 40 checksum: ['0FE94DDFAA3EC01E3F53F8127BD5235C'] OK
common_key: 50 checksum: ['D94617F22C8CDFD888D8C45C10E34B13'] OK
common_key: 60 checksum: ['5C594C0787E99AB17C54E72A1410F935'] OK
common_key: 70 checksum: ['C65527EDA25DBC6FACE68D0236706125'] OK
common_key: 80 checksum: ['25DE08A1CE68B51D18056F4DED240818'] OK
common_key: 90 checksum: ['D9B0ABAD319ADD9F67450EE5E6E689B4'] OK
common_key: 100 checksum: ['D8F4AF0496652FF9EDCB224CC1A45905'] OK
     spec.common_key spec.algo_name                   result.checksum
125               50      btree1024  D94617F22C8CDFD888D8C45C10E34B13
187               50      btree1024  D94617F22C8CDFD888D8C45C10E34B13
224               50      btree1024  D94617F22C8CDFD888D8C45C10E34B13
284               50      btree1024  D94617F22C8CDFD888D8C45C10E34B13